# Transform the data to work with Snorkel: Part 1 - Event Type

Essentially we will have to create two labeling models.
One assigns labels to event types and the other assigns labels to argument roles in event mentions.

In any case we need to create a row for each event (trigger) to do event type labeling.

For this we need 1 additional column:
- trigger_id

One numpy array containing the:
- event_type

We will probably focus on keyword lists and some heuristics to create our labeling functions.

In [ ]:
import sys
sys.path.append("../")
import warnings
import pandas as pd
import numpy as np
from wsee.utils import utils
from wsee.data import pipeline

In [ ]:
warnings.filterwarnings(action='once')
pd.set_option('display.max_colwidth', None)
DATA_DIR = '/Users/phuc/data/daystream_first_trigger_check'  # replace path to corpus

### SD4M Event Types

| Number | Code                   | Description                                                                             |
|--------|------------------------|-----------------------------------------------------------------------------------------|
| -1     | ABSTAIN                | No vote, for Labeling Functions                                                         |
| 0      | Accident               | Collision of a vehicle with another vehicle, person, or obstruction                     |
| 1      | CanceledRoute          | Cancellation of public transport routes                                                 |
| 2      | CanceledStop           | Cancellation of public transport stops                                                  |
| 3      | Delay                  | Delay resulting from remaining traffic disturbances                                     |
| 4      | Obstruction            | Temporary installation to control traffic                                               |
| 5      | RailReplacementService | Replacement of a passenger train by buses or other substitute public transport services |
| 6      | TrafficJam             | Line of stationary or very slow-moving traffic                                          |
| 7      | O                      | No SD4M event.                                                                          |

In [ ]:
loaded_data = pipeline.load_data(DATA_DIR)
sd_train = loaded_data['train']
sd_dev = loaded_data['dev']
sd_test = loaded_data['test']

daystream = loaded_data['daystream']

In [ ]:
sd_train.head()

## Step 1: Create one row for every event trigger

We will use the (labeled) SD4M training set as our development data to create our labeling functions.
In this notebook we will run our labeling functions and our LabelModel on that data.
In the real pipeline we will instead label the Daystream data that does not have event type and event argument role labels.

In [ ]:
df_dev, Y_dev = pipeline.build_event_trigger_examples(sd_train)

We use the (labeled) SD4m development set as our "test set" to measure the performance of our LabelModel.

In [ ]:
df_test, Y_test = pipeline.build_event_trigger_examples(sd_dev)

In [ ]:
from wsee import SD4M_RELATION_TYPES
print(SD4M_RELATION_TYPES)

## Step 2: Explore the data

In [ ]:
from wsee.preprocessors.preprocessors import *
from wsee.data import explore, pipeline

We can apply all our preprocessors on our data and see if we can find something interesting for our labeling functions.
Let's first sample the SD4M training data, which is labeled.

In [ ]:
labeled_sd4m_triggers = explore.add_labels(df_dev, Y_dev)

In [ ]:
labeled_sd4m_triggers = explore.apply_preprocessors(labeled_sd4m_triggers, [pre_trigger_left_tokens, pre_mixed_ner, pre_trigger_right_tokens])

In [ ]:
labeled_sd4m_triggers = explore.add_event_types(labeled_sd4m_triggers)

Now we can collect the trigger words per class.

In [ ]:
n = 100
filtered_sd4m_triggers = labeled_sd4m_triggers[labeled_sd4m_triggers['label'] != 7]
print(f"Number of events: {len(labeled_sd4m_triggers)}\n")
for idx, class_name in enumerate(SD4M_RELATION_TYPES):
    class_sd4m_triggers = labeled_sd4m_triggers[labeled_sd4m_triggers['label'] == idx]
    print(f"{class_name}: {len(class_sd4m_triggers)} instances")

## Step 3: Evaluate the labeling functions on the SD4M training data

In [ ]:
from wsee.labeling import event_trigger_lfs as trigger_lfs

In [ ]:
from snorkel.labeling import PandasLFApplier

lfs = [
    trigger_lfs.lf_accident_context,
    trigger_lfs.lf_accident_context_street,
    trigger_lfs.lf_accident_context_no_cause_check,
    trigger_lfs.lf_canceledroute_cat,
    trigger_lfs.lf_canceledroute_amplifier1,
    trigger_lfs.lf_canceledstop_cat,
    trigger_lfs.lf_canceledstop_amplifier1,
    trigger_lfs.lf_delay_cat,
    trigger_lfs.lf_delay_priorities,
    trigger_lfs.lf_delay_duration,
    trigger_lfs.lf_obstruction_cat,
    trigger_lfs.lf_obstruction_street,
    trigger_lfs.lf_obstruction_priorities,
    trigger_lfs.lf_railreplacementservice_cat,
    trigger_lfs.lf_railreplacementservice_amplifier1,
    trigger_lfs.lf_trafficjam_cat,
    trigger_lfs.lf_trafficjam_street,
    trigger_lfs.lf_trafficjam_order,
    trigger_lfs.lf_negative,
    trigger_lfs.lf_cause_negative,
    trigger_lfs.lf_obstruction_negative
]

applier = PandasLFApplier(lfs)

In [ ]:
L_dev = applier.apply(df_dev)
L_test = applier.apply(df_test)

In [ ]:
from snorkel.labeling import LFAnalysis

LFAnalysis(L_dev, lfs).lf_summary(Y_dev)

## Step 4: Error Analysis 
Now we can look at the LabelMatrix for errors. We need to use the DataFrame from the exploration section, which includes the information from the preprocessors.
We can then specifically look for the instances that were labeled incorrectly.

We will first look at the keyword based labeling function for accidents:

In [ ]:
from wsee.labeling import error_analysis
relevant_cols = ['text','trigger', 'event_types']

In [ ]:
labeled_sd4m_triggers.iloc[L_dev[:, 3] == 1].sample(5)[['text', 'trigger', 'label']]

In [ ]:
with pd.option_context('display.max_rows', None):  # more options can be specified also
    display(error_analysis.get_false_positives(labeled_df=labeled_sd4m_triggers, lf_outputs=L_dev, lf_index=3, label_of_interest=1)[relevant_cols])

In [ ]:
error_analysis.get_abstained_instances(labeled_df=labeled_sd4m_triggers, lf_outputs=L_dev, lf_index=9, label_of_interest=4)[relevant_cols]

## Step 5: Train the Labeling model and label the data

In [ ]:
sd4m_trigger_class_balance = [
    0.07221542227662178,
    0.07466340269277846,
    0.030599755201958383,
    0.0795593635250918,
    0.12362301101591187,
    0.02692778457772338,
    0.189718482252142,
    0.40269277845777235
]

daystream_trigger_lf_class_balance = [
    0.05769230769230769,
    0.24038461538461536,
    0.028846153846153844,
    0.2019230769230769,
    0.19230769230769232,
    0.09615384615384616,
    0.028846153846153844,
    0.15384615384615385
]

In [ ]:
from snorkel.labeling import LabelModel

label_model = LabelModel(cardinality=8, verbose=True)
label_model.fit(L_train=L_dev, n_epochs=500, log_freq=100, seed=123, class_balance=sd4m_trigger_class_balance)

In [ ]:
label_model_acc = label_model.score(L=L_dev, Y=Y_dev, tie_break_policy="random")[
    "accuracy"
]
print(f"{'Label Model Accuracy:':<25} {label_model_acc * 100:.1f}%")

In [ ]:
probs_train = label_model.predict_proba(L=L_dev)

In the proposed workflow one would filter out all the datapoints that were not labeled by any of the labeling functions.
We will follow this approach as that does not affect the merging process in our Snorkel processing pipeline.
While it may result in sentences missing certain events, they would then be processed as dummy events in the AllenNLP model and factored out during the loss calculation (?).

In [ ]:
from snorkel.labeling import filter_unlabeled_dataframe

df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(
    X=df_dev, y=probs_train, L=L_dev
)

In the Snorkel processing pipeline we would merge the labeled dataframes back together that belong to the same document and proceed with labeling the event argument roles.

In [ ]:
labeled_sd_train = pipeline.merge_event_trigger_examples(df_train_filtered.drop('event_triggers', axis=1), probs_train_filtered)
labeled_sd_train.reset_index(level=0).to_json(DATA_DIR + "/save_sd_triggers.jsonl", orient='records', lines=True, force_ascii=False)

## Step 6: Label the Daystream data with Snorkel

In [ ]:
df_train, Y_train = pipeline.build_event_trigger_examples(daystream)
L_train = applier.apply(df_train)

In [ ]:
LFAnalysis(L_train, lfs).lf_summary()

In [ ]:
daystream_model = LabelModel(cardinality=8, verbose=True)
daystream_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=123, class_balance=daystream_trigger_lf_class_balance)

In [ ]:
daystream_model_acc = daystream_model.score(L=L_test, Y=Y_test, tie_break_policy="random")[
    "accuracy"
]
print(f"{'Label Model Accuracy:':<25} {daystream_model_acc * 100:.1f}%")

In [ ]:
daystream_probs = daystream_model.predict_proba(L=L_train)

In [ ]:
from snorkel.labeling import filter_unlabeled_dataframe

df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(
    X=df_train, y=daystream_probs, L=L_train
)

In [ ]:
labeled_daystream = pipeline.merge_event_trigger_examples(df_train_filtered, probs_train_filtered)

In [ ]:
labeled_daystream.reset_index(level=0).to_json(DATA_DIR + "/save_daystreamv6_triggers.jsonl", orient='records', lines=True, force_ascii=False)

## Step 7: Check Daystream Labeling

In [ ]:
df_train['trigger_probs'] = list(daystream_probs)
df_train['most_probable_class'] = [SD4M_RELATION_TYPES[label_idx] for label_idx in daystream_probs.argmax(axis=1)]
df_train['max_class_prob'] = ["{:.2f}".format(class_prob) for class_prob in daystream_probs.max(axis=1)]

In [ ]:
for trigger_class in SD4M_RELATION_TYPES:
    print(f"{trigger_class}: {len(df_train[df_train['most_probable_class'] == trigger_class])} instances")

Code to display all the rows of the dataframe:
```python
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_train[df_train['most_probable_class'] == 'O'][['text', 'trigger', 'most_probable_class', 'max_class_prob', 'trigger_probs']])
```

In [ ]:
df_train[df_train['most_probable_class'] == 'Obstruction'].sample(10)[['text', 'trigger', 'most_probable_class', 'max_class_prob', 'trigger_probs']]